In [1]:
import random
import math
import numpy as np

In [3]:
class Data:

    # initialize needed values
    def __init__(self, address):
        self.name = ""
        self.num_cities = 0
        self.locations = []
        self.address = address

    # read test case
    def load_data(self):
        with open (self.address, 'r') as f:
            self.name = f.readline().split(" ")[1]
            self.num_cities = int(self.name[2:])
            f.readline()
            f.readline()
            f.readline()
            f.readline()
            f.readline()
            f.readline()
            for _ in range(self.num_cities):
                city = list(map(float, f.readline().split(" ")))[1:]
                self.locations.append(city)


In [76]:
class Gene:

    # initialize parameters
    def __init__(self, num_cities, locations):
        self.gene = []
        self.num_cities = num_cities
        self.locations = locations
        self.fitness = math.inf


    # create random gene for first population
    def initial_gene(self):
        self.gene = [i for i in range(self.num_cities)]
        random.shuffle(self.gene)

    # calculate fitness 
    def fitness_function(self):
        self.fitness = 0
        for i in range(self.num_cities-1):
            self.fitness += np.sqrt((self.locations[self.gene[i]][0] - self.locations[self.gene[i+1]][0])**2 + (self.locations[self.gene[i]][1] - self.locations[self.gene[i+1]][1])**2)
        return self.fitness
    
    # mutate gene
    def mutation(self):
        i = random.randint(1, self.num_cities-1)
        j = random.randint(1, self.num_cities-1)

        # randomly replace 2 positions in gene
        tmp = self.gene[i]
        self.gene[i] = self.gene[j]
        self.gene[j] = tmp

        self.fitness = self.fitness_function()

    # define a local search to search for neighbors of a gene that is better than itself
    def local_search(self, num_neighbors):

        neighbors = []
        for _ in range(num_neighbors):
            genei = Gene(self.num_cities, self.locations)
            genei.gene = self.gene
            i = random.randint(1, self.num_cities-1)
            j = random.randint(1, self.num_cities-1)

            # randomly change 2 positions in gene
            tmp = genei.gene[i]
            genei.gene[i] = genei.gene[j]
            genei.gene[j] = tmp
            
            genei.fitness = genei.fitness_function()
            neighbors.append((genei.fitness, genei))

        # select best neighbor
        neighbors.append((self.fitness, self))
        neighbors = sorted(neighbors, key=lambda x: x[0])
        
        return neighbors[0][1]


In [77]:
class Population:

    # initialize population's parameters
    def __init__(self, data, pop_size=100):
        self.population_size  = pop_size
        self.population       = []
        self.locations        = data.locations
        self.num_cities       = data.num_cities


    # generate first population randomly
    def initialize_population(self):
        for _ in range(self.population_size):
            genei = Gene(self.num_cities, self.locations)
            genei.initial_gene()
            self.population.append(genei)

    # generate first population starting from each city
    def initialize_population_one_city(self):
        for i in range(self.population_size):
            genei         = Gene(self.num_cities, self.locations)
            genei.initial_gene()
            genelist      = genei.gene.copy()
            ind           = genelist.index(i)
            tmp           = genelist[ind]
            genelist[ind] = genelist[0]
            genelist[0]   = tmp
            genei.gene    = genelist
            self.population.append(genei)
    

    # selest parents from population for crossover using tournament selection
    def parent_selection(self, N=3):
        # choose random N candidates to attend tornoment
        father_candidates = [random.choice(self.population) for _ in range(N)]
        mother_candidates = [random.choice(self.population) for _ in range(N)]

        # choose the best one of N candidates to be a parent
        # find candidates fitnesses
        father_vals = [(father.fitness, father) for father in father_candidates]
        mother_vals = [(mother.fitness, mother) for mother in mother_candidates]
        # sort candidates
        sorted_fathers = sorted(father_vals, key=lambda x: x[0])
        sorted_mothers = sorted(mother_vals, key=lambda x: x[0])
        # select parents with min fitness
        best_father = sorted_fathers[0][1]
        best_mother = sorted_mothers[0][1]
        
        return best_father, best_mother


    # crossover function
    def crossover(self, father, mother):
        father = father.gene
        mother = mother.gene
        child1l, child2l = [], []
        child1, child2 = Gene(self.num_cities, self.locations), Gene(self.num_cities, self.locations)
        random_point = random.randint(1, self.num_cities-1)    

        for i in range(0, random_point):
                child1l.append(father[i])
                child2l.append(mother[i])

        for i in range(random_point, self.num_cities):         # add the rest of other parent from random point if its not used before from previous parent
            if mother[i] not in child1l:   
                child1l.append(mother[i])
            
            if father[i] not in child2l:
                child2l.append(father[i])

        for i in range(0, random_point):     # add other parent from first to the random point
            if mother[i] not in child1l:
                child1l.append(mother[i])

            if father[i] not in child2l:
                child2l.append(father[i])
                
        child1.gene    = child1l
        child1.fitness = child1.fitness_function()  
        child2.gene    = child2l 
        child2.fitness = child2.fitness_function() 
        return child1, child2
    



In [49]:
def genetic_algorithm_main(test_address, generation_num, p_cross, p_mutate, p_replace, pop_size, tornoment_N, expected_value):
    best_cost = math.inf
    best_result = []
    # load test case
    tsp_test1 = Data(test_address)
    tsp_test1.load_data()

    # create first population
    population = Population(tsp_test1, pop_size=pop_size)
    population.initialize_population()

    for g_num in range(generation_num):
        if best_cost < expected_value:
            break
        # crossover and create as many children as parents population
        children_population = Population(tsp_test1, pop_size=pop_size)
        counter = 0
        while counter < pop_size:
            # select 2 parents for crossover
            father, mother = population.parent_selection(N=tornoment_N)

            # crossover 2 selected parents with p_cross
            e_cross = random.random()
            if e_cross < p_cross:
                child1, child2 = population.crossover(father, mother)
                counter += 2

                # mutate childs with p_mutate
                e_mutate1 = random.random()
                if e_mutate1 < p_mutate:
                    child1.mutation()
                e_mutate2 = random.random()
                if e_mutate2 < p_mutate:
                    child2.mutation()

                children_population.population.append(child1)
                children_population.population.append(child2)
        
        # population replacement
        parent_values = []
        child_values = []
        for i in range(pop_size):             # save all values of parents and children in 2 value lists
            x = population.population[i].fitness
            y = children_population.population[i].fitness
            parent_values.append(x)
            child_values.append(y)

        num_replace = p_replace * pop_size    # find the number of parents that should be replaced with children in population

        for i in range(int(num_replace)):        # replace as the size of replacements in population
            max_parent_ind = parent_values.index(max(parent_values))  # find the worst parent
            min_child_ind = child_values.index(min(child_values))      # find best children

            population.population.pop(max_parent_ind)      # remove worst parent from population
            population.population.append(children_population.population[min_child_ind])  # add best children to population

            children_population.population.pop(min_child_ind)  # remove best children from population of children
            parent_values.pop(max_parent_ind)   # remove worst parent value from parents values
            child_values.pop(min_child_ind)  # remove best children value from children values

        # find the best result in population
        for i in range(pop_size):
            if population.population[i].fitness < best_cost:
                best_cost = population.population[i].fitness
                best_result = population.population[i].gene.copy()

        print(f"generation {g_num}, best_cost so far: {best_cost}")

    return best_result, tsp_test1

            


In [ ]:
best_result, test = genetic_algorithm_main(test_address="TSP-Tests/gr229.tsp", generation_num=100000, p_cross=0.65, p_mutate=0.6, p_replace=0.1, pop_size=100, tornoment_N=3, expected_value=2500)

In [78]:
def memetic_algorithm_main(test_address, generation_num, p_cross, p_mutate, p_neighbor, p_replace, pop_size, tornoment_N, expected_value):
    best_cost = math.inf
    best_result = []
    # load test case
    tsp_test1 = Data(test_address)
    tsp_test1.load_data()

    pop_size = tsp_test1.num_cities
    # create first population
    # population = Population(tsp_test1, pop_size=pop_size)
    population = Population(tsp_test1, pop_size=pop_size)
    population.initialize_population_one_city()
    # for i in range(pop_size):
    #     print(population.population[i].gene)
    # population.initialize_population()

    for g_num in range(generation_num):
        if best_cost < expected_value:
            break
        # crossover and create as many children as parents population
        children_population = Population(tsp_test1, pop_size=pop_size)
        # children_population = Population(tsp_test1, pop_size=pop_size)
        counter = 0
        while counter < pop_size:
            # select 2 parents for crossover
            father, mother = population.parent_selection(N=tornoment_N)

            # crossover 2 selected parents with p_cross
            e_cross = random.random()
            if e_cross < p_cross:
                child1, child2 = population.crossover(father, mother)
                counter += 2

                # mutate childs with p_mutate
                e_mutate1 = random.random()
                if e_mutate1 < p_mutate:
                    child1.mutation()
                e_mutate2 = random.random()
                if e_mutate2 < p_mutate:
                    child2.mutation()

                # do local search on chilren's neighbors
                e_neighbor1 = random.random()
                if e_neighbor1 < p_neighbor:
                    genei = child1.local_search(num_neighbors=10)
                else:
                    genei = child1
                e_neighbor2 = random.random()
                if e_neighbor2 < p_neighbor:
                    genej = child2.local_search(num_neighbors=10)
                else:
                    genei = child2

                children_population.population.append(child1)
                children_population.population.append(child2)
        
        # population replacement
        parent_values = []
        child_values = []
        for i in range(pop_size):             # save all values of parents and children in 2 value lists
            x = population.population[i].fitness
            y = children_population.population[i].fitness
            parent_values.append(x)
            child_values.append(y)

        num_replace = p_replace * pop_size    # find the number of parents that should be replaced with children in population

        for i in range(int(num_replace)):        # replace as the size of replacements in population
            max_parent_ind = parent_values.index(max(parent_values))  # find the worst parent
            min_child_ind = child_values.index(min(child_values))      # find best children

            population.population.pop(max_parent_ind)      # remove worst parent from population
            population.population.append(children_population.population[min_child_ind])  # add best children to population

            children_population.population.pop(min_child_ind)  # remove best children from population of children
            parent_values.pop(max_parent_ind)   # remove worst parent value from parents values
            child_values.pop(min_child_ind)  # remove best children value from children values

        # find the best result in population
        for i in range(pop_size):
            if population.population[i].fitness < best_cost:
                best_cost = population.population[i].fitness
                best_result = population.population[i].gene.copy()

        print(f"generation {g_num}, best_cost so far: {best_cost}")

    return best_result, tsp_test1

            


In [82]:
best_result, test = memetic_algorithm_main(test_address="TSP-Tests/pr1002.tsp", generation_num=500, p_cross=0.65, p_mutate=0.5, p_neighbor=0.5, p_replace=0.1, pop_size=100, tornoment_N=5, expected_value=2500)

generation 0, best_cost so far: 6138178.101429166
generation 1, best_cost so far: 6138178.101429166
generation 2, best_cost so far: 6040086.145148863
generation 3, best_cost so far: 5989505.858291098
generation 4, best_cost so far: 5965061.1518393
generation 5, best_cost so far: 5935108.193463841
generation 6, best_cost so far: 5897462.322136448
generation 7, best_cost so far: 5837240.462100838
generation 8, best_cost so far: 5822009.559247483
generation 9, best_cost so far: 5790717.5234725345
generation 10, best_cost so far: 5756722.970047513
generation 11, best_cost so far: 5742888.750011781
generation 12, best_cost so far: 5723494.43098761
generation 13, best_cost so far: 5696791.200616215
generation 14, best_cost so far: 5667753.252947812
generation 15, best_cost so far: 5630555.377464153
generation 16, best_cost so far: 5606092.259768214
generation 17, best_cost so far: 5583696.443613503
generation 18, best_cost so far: 5559837.309754811
generation 19, best_cost so far: 5531567.88

KeyboardInterrupt: 